In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, get_dummies, concat
import requests
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import train_test_split
from sklearn.metrics import r2_score
from ggplot import *
%matplotlib inline 
import twitter

In [3]:
# predict on year, day of month optionally -- those are available but for some reason we aren't modeling on those - of course, predicting on previous values would also be nice but not sure how to integrate this into forecasting

In [664]:
class forecast(object):    
    
    def __init__(self, forestColumnNames, frst):
        api = twitter.Api(consumer_key='hjnVjCeTGhG9ReoG35MyDerx1',consumer_secret='oPTjLl5nEBWqVfxGC5Db04uDWzzhVVh3hRUb643jmkZrn7zhIW', access_token_key='3173034274-Lj194JwBIoHIAMQDrUXvZw2YlFNsgNMYd4xVz1R',access_token_secret='pLQbEnbbr8YJRFmMvfoWdVwWwBfYN0UjSrBOHA01yC39M')
        self.forestColumnNames = forestColumnNames
        self.frst = frst
        API_KEY = 'a3d4db71573f30f9'
        url = 'http://api.wunderground.com/api/'+API_KEY+'/hourly10day/q/Beijing/Beijing.json'
        content = requests.get(url).json()
        self.raw = content['hourly_forecast']
        self.df = self.prepForRF()
        
    def day_to_int(self, day_string):
        dayvals = {
            'Sunday': 0,
            'Monday': 1,
            'Tuesday': 2,
            'Wednesday': 3,
            'Thursday': 4, 
            'Friday': 5, 
            'Saturday': 6
        }
        return dayvals.get(day_string, 'nothing')
        
    def extractRow(self, one):   
        date = one['FCTTIME']
        return dict(year=date['year'], month=date['mon'], day=date['mday'], wday=self.day_to_int(date['weekday_name']), hour=date['hour'], pressurei=one['mslp']['english'], icon=one['icon'], dewpti=one['dewpoint']['english'], hum=one['humidity'], temp=one['temp']['metric'], wdird=one['wdir']['degrees'], wdire=one['wdir']['dir'], wspdm=one['wspd']['metric'])
         
    def extractRows(self):
        return DataFrame([self.extractRow(x) for x in self.raw])
    
    # make into binary values categorical vars
    def categorizeVars(self):
        df = self.extractRows()
        return df
        #return pd.concat([df, get_dummies(df.wdire), get_dummies(df.icon)], axis=1)
        #return pd.concat([df, get_dummies(df.icon)], axis=1)

    
    def prepForRF(self):
        df = self.categorizeVars()
        # rename certain vars, E to East, W to West, Month to mon, because of inconsistencies in the wunderground API
        #df = df.rename(columns={'E':'East', 'W':'West', 'N':'North', 'S':'South', 'month': 'mon', 'temp' : 'tempm'})
        df = df.rename(columns={'month': 'mon', 'temp' : 'tempm'})
        # drop icon, wdire
        # later rebuild the model with day and year
        df = df.drop(['icon', 'wdire', 'day', 'year', 'hour', 'mon'], axis=1)
        df = self.imputeVals(df)
        prev_pol = self.get_prev_pol()
        df = self.addPredictions(df, prev_pol)
        return df
    
    def get_prev_pol(self):
        tweets = [s.text for s in api.GetUserTimeline(screen_name='@BeijingAir')]
        prev_vals = [int(x.split(';')[-2]) for x in tweets][0:5]
        # first row only
        #df.loc[0,['nminus1Val', 'nminus2Val', 'nminus3Val', 'nminus4Val', 'nminus5Val']] = prev_vals
        return prev_vals
    
    def imputeVals(self, df):
        # any missing values in the test frame need to be imputed with all zeroes
        cols_to_add = self.forestColumnNames - df.columns
        for (idx, val) in enumerate(cols_to_add):
            df[val] = np.zeros(df.shape[0])
        return df
    
    def addPredictions(self, df, prev_pol):
        
        #make a column of zeros
        predictions = np.zeros(len(df))
        
        values = prev_pol
        
        preds = []
        
        # reorder so that in the same shape as the model
        df = df[self.forestColumnNames]

        for i in xrange(len(df)):            

            # fill row
            df.loc[i, ['nminus1Val', 'nminus2Val', 'nminus3Val', 'nminus4Val', 'nminus5Val']] = values

            # make a prediction
            prediction = self.frst.predict(df.ix[i])

            # update prediction column
            #df.loc[i, 'prediction'] = prediction
            #preds.append(prediction)            

            # add prediction to front
            values.insert(0, prediction.item())
            
            # drop last value
            values.pop()
            
            preds.append(prediction.item())
        
        df['predictions'] = preds
        return df
    
    

In [665]:
bz = forecast(forest_.X_test.columns, forest_.rf)

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 jobs     

In [666]:
bz.df.head()

,dewpti,hum,mday,nminus1Val,nminus2Val,nminus3Val,nminus4Val,nminus5Val,pressurei,tempm,wday,wdird,wspdm,predictions
0,40,78,0,119.00,97.00,54.00,95.00,80,30.16,8,2,339,8,138.52
1,39,76,0,138.52,119.00,97.00,54.00,95,30.16,8,2,342,6,139.12
2,39,70,0,139.12,138.52,119.00,97.00,54,30.17,9,2,343,10,133.43
3,38,69,0,133.43,139.12,138.52,119.00,97,30.18,9,2,342,8,130.28
4,39,65,0,130.28,133.43,139.12,138.52,119,30.19,10,2,342,11,128.49


In [658]:
pd.DataFrame(bz.df.iloc[0]).T

,dewpti,hum,pressurei,tempm,wday,wdird,wspdm,mday,nminus1Val,nminus2Val,nminus3Val,nminus4Val,nminus5Val
0,42,75,30.16,10,2,339,5,0,97,54,95,80,82


In [659]:
tst = pd.DataFrame({'dewpti':42, 'hum':75, 'pressurei':30.16, 'tempm':10, 'wday':2, 'wdird':339, 'wspdm':5, 'mday':0, 'nminus1Val': 97, 'nminus2Val':54, 'nminus3Val':95, 'nminus4Val':80, 'nminus5Val': 82}, index=[0])

In [661]:
tst

,dewpti,hum,mday,nminus1Val,nminus2Val,nminus3Val,nminus4Val,nminus5Val,pressurei,tempm,wday,wdird,wspdm
0,42,75,0,97,54,95,80,82,30.16,10,2,339,5


In [663]:
forest_.X_test.head()

,dewpti,hum,mday,nminus1Val,nminus2Val,nminus3Val,nminus4Val,nminus5Val,pressurei,tempm,wday,wdird,wspdm
47995,51.8,56,21,106,114,122,121,45,29.71,20,3,320,10.8
20096,-0.4,57,22,67,61,44,43,51,30.39,-11,6,310,7.2
57216,57.2,33,15,81,76,72,73,80,29.65,32,1,350,25.2
1419,62.6,88,7,108,94,80,82,79,29.71,19,6,340,7.2
32111,73.4,94,10,52,61,73,66,62,29.50,24,2,360,3.6


In [660]:
forest_.rf.predict(tst)

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


array([ 102.84])

In [635]:
forest_.X_test['predictions'] = forest_.rf.predict(frT.X_test)
forest_.X_test['actual'] = forest_.y_test
forest_.X_test

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.6s finished


,dewpti,hum,mday,nminus1Val,nminus2Val,nminus3Val,nminus4Val,nminus5Val,pressurei,tempm,wday,wdird,wspdm,predictions,actual
47995,51.8,56,21,106,114,122,121,45,29.71,20,3,320.00000,10.8,101.16,62
20096,-0.4,57,22,67,61,44,43,51,30.39,-11,6,310.00000,7.2,76.87,88
57216,57.2,33,15,81,76,72,73,80,29.65,32,1,350.00000,25.2,104.72,65
1419,62.6,88,7,108,94,80,82,79,29.71,19,6,340.00000,7.2,108.65,108
32111,73.4,94,10,52,61,73,66,62,29.50,24,2,360.00000,3.6,50.45,48
19180,12.2,63,12,112,131,109,119,122,30.01,-5,0,350.00000,3.6,112.29,115
43519,28.4,53,13,185,160,157,163,155,30.18,7,3,300.00000,3.6,194.12,189
35039,23.0,49,16,19,20,25,20,17,30.15,5,5,333.20694,14.4,18.87,23
27097,14.0,33,7,17,28,27,146,202,30.51,5,3,320.00000,36.0,15.05,18
22669,42.8,71,22,74,58,53,47,55,29.92,11,0,290.00000,7.2,82.12,122


In [440]:
class forest():
    
    def __init__(self):
        data = pd.read_csv('data_newer.csv')
        # the first 5 rows have NAs
        data = data.ix[5:,]
        # just in case other nulls
        data = data.fillna(data.mean())
        #data=pd.read_csv('data.csv')
        self.X_train, self.X_test, self.y_train, self.y_test = self.prepData(data)
        self.rf = self.buildForest(self.X_train, self.y_train)
        #self.measures = plotMetrics()
        
    def prepData(self, df):
        df = self.catVars(df)
        df = self.removeUnusedVars(df)
        self.df = df
        return self.splitSet(df)
        
    def catVars(self, df):
        wdire_cat = get_dummies(df.wdire)
        icon_cat = get_dummies(df.icon)
        #return concat([df, wdire_cat, icon_cat], axis=1)
        #return concat([df, icon_cat], axis=1)
        return df
    
    # would like to drop the cat(wind direction in favor of the numerical variety)
    def removeUnusedVars(self, df):
        return df.drop(['conds', 'icon', 'visi', 'wdire', 'Unnamed: 0', 'year'], axis=1)
        #return df.drop(['conds', 'icon', 'visi', 'wdire', 'Unnamed: 0'], axis=1)
    
    def splitSet(self, df):
        # create training (80%) and test (20%) sets
        X_train, X_test, y_train, y_test = train_test_split(df[df.columns.difference(['Value'])], df.Value, test_size=0.2, random_state=42)
        return X_train, X_test, y_train, y_test
    
    def buildForest(self, X_train, y_train):
        rf = RandomForestRegressor(n_estimators=100, verbose=1)
        rf.fit_transform(X_train, y_train)
        return rf
    
    def metrics(self, y_test, X_test, rf):
        self.r2 = r2_score(y_test, rf.predict(X_test))
        self.mse = np.mean((y_test - rf.predict(X_test))**2)
        self.rmse = np.sqrt(mse)
    
    def plotMetrics(self):
        measures = DataFrame({"feature_importances_": self.rf.feature_importances_, "names" : X_train.columns})
        ggplot(measures, aes(x='names', y='feature_importances_')) + geom_bar(stat='bar') + theme(axis_text_x = element_text(angle = 90, hjust = 1))
        
        

In [647]:
forest_ = forest()
frT = forest_
print frT.rf.score(frT.X_test, frT.y_test)
mse = np.sqrt(np.mean((forest_.y_test - forest_.rf.predict(forest_.X_test))**2))
print mse
#pd.isnull(forest_.X_train).any(axis=1)
#forest_.X_train[pd.isnull(forest_.X_train).any(axis=1)].T

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.5s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:   25.5s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   51.8s finished
[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:    0.2s


0.932880156224
22.7420045335


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.5s finished


In [20]:
frcast = forecast(fr.X_train.columns, fr.rf)

//anaconda/lib/python2.7/site-packages/pandas/core/index.py:1353: FutureWarning: using '-' to provide set differences with Indexes is deprecated, use .difference()
  "use .difference()",FutureWarning)
[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 200 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 450 jobs       | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    0.1s finished


In [31]:
ggplot(measures, aes(x='names', y='feature_importances_')) + geom_bar(stat='bar') + theme(axis_text_x = element_text(angle = 90, hjust = 1))

NameError: name 'measures' is not defined

In [95]:
import cPickle

with open('rforestModel.cpickle', 'wb') as f:
    cPickle.dump(frT.rf, f)

In [94]:
frT = forest_
frT.rf.score(frT.X_test, frT.y_test)

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.2s finished


0.71631081170597599

In [304]:
frT = forest_
frT.rf.score(frT.X_test, frT.y_test)

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.3s finished


0.931614365024515

In [462]:
fr = forest_
measures = concat([DataFrame({"name":fr.X_test.columns}), DataFrame({"importance":fr.rf.feature_importances_})], axis=1)

In [394]:
mse = np.sqrt(np.mean((forest_.y_test - forest_.rf.predict(forest_.X_test))**2))

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.5s finished


In [395]:
np.sqrt(mse)

4.765665553740253

In [493]:
measures.sort("importance", ascending=False)

,name,importance
3,nminus1Val,0.930332
4,nminus2Val,0.011793
5,nminus3Val,0.007725
0,dewpti,0.006981
12,wspdm,0.006231
7,nminus5Val,0.006037
6,nminus4Val,0.005588
1,hum,0.005429
8,pressurei,0.004955
9,tempm,0.004765


In [496]:
measures

,name,importance
0,dewpti,0.006981
1,hum,0.005429
2,mday,0.004730
3,nminus1Val,0.930332
4,nminus2Val,0.011793
5,nminus3Val,0.007725
6,nminus4Val,0.005588
7,nminus5Val,0.006037
8,pressurei,0.004955
9,tempm,0.004765


In [499]:
measures.iloc[:,-2]

0         dewpti
1            hum
2           mday
3     nminus1Val
4     nminus2Val
5     nminus3Val
6     nminus4Val
7     nminus5Val
8      pressurei
9          tempm
10          wday
11         wdird
12         wspdm
Name: name, dtype: object

In [68]:
frT.rf.score(frT.X_test, frT.y_test)

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done 200 jobs       | elapsed:    1.1s
[Parallel(n_jobs=1)]: Done 450 jobs       | elapsed:    2.4s
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    2.6s finished


0.72274540476990845

In [76]:
np.mean(fr.rf.feature_importances_)

0.027777777777777773

In [78]:
frT.rf.feature_importances_

array([  2.94917129e-03,   6.28787879e-03,   5.06512666e-03,
         2.29022786e-03,   4.30306659e-03,   3.57140091e-03,
         1.80284268e-03,   7.27357838e-03,   6.82429414e-03,
         5.83397796e-03,   3.98994528e-03,   1.61447589e-03,
         6.11637599e-03,   5.29339914e-03,   1.46440063e-03,
         5.96633894e-04,   1.19962871e-03,   3.17849175e-01,
         4.18897450e-03,   7.81169674e-02,   4.14882214e-03,
         7.89791295e-02,   6.04215625e-02,   8.12641938e-02,
         5.12873060e-02,   1.18620679e-03,   2.05567424e-03,
         7.80871288e-02,   8.24824708e-03,   8.89212949e-06,
         8.19644046e-04,   5.30013065e-02,   1.89028393e-03,
         4.40651675e-03,   4.86040254e-02,   5.89595185e-02])

In [81]:
fr.rf.transform(fr.X_train).shape

(44999, 10)

In [82]:
fr.X_train.shape

(44999, 36)

In [97]:
API_KEY = 'a3d4db71573f30f9'
url = 'http://api.wunderground.com/api/'+API_KEY+'/hourly10day/q/Beijing/Beijing.json'
content = requests.get(url).json()
raw = content['hourly_forecast']

In [98]:
raw

[{u'FCTTIME': {u'UTCDATE': u'',
   u'age': u'',
   u'ampm': u'AM',
   u'civil': u'8:00 AM',
   u'epoch': u'1444608000',
   u'hour': u'8',
   u'hour_padded': u'08',
   u'isdst': u'0',
   u'mday': u'12',
   u'mday_padded': u'12',
   u'min': u'00',
   u'min_unpadded': u'0',
   u'mon': u'10',
   u'mon_abbrev': u'Oct',
   u'mon_padded': u'10',
   u'month_name': u'October',
   u'month_name_abbrev': u'Oct',
   u'pretty': u'8:00 AM CST on October 12, 2015',
   u'sec': u'0',
   u'tz': u'',
   u'weekday_name': u'Monday',
   u'weekday_name_abbrev': u'Mon',
   u'weekday_name_night': u'Monday Night',
   u'weekday_name_night_unlang': u'Monday Night',
   u'weekday_name_unlang': u'Monday',
   u'yday': u'284',
   u'year': u'2015'},
  u'condition': u'Clear',
  u'dewpoint': {u'english': u'39', u'metric': u'4'},
  u'fctcode': u'1',
  u'feelslike': {u'english': u'49', u'metric': u'9'},
  u'heatindex': {u'english': u'-9999', u'metric': u'-9999'},
  u'humidity': u'67',
  u'icon': u'clear',
  u'icon_url': u'h

In [335]:
import twitter

In [ ]:
twitter.Api.g

In [337]:
api = twitter.Api(consumer_key='hjnVjCeTGhG9ReoG35MyDerx1',consumer_secret='oPTjLl5nEBWqVfxGC5Db04uDWzzhVVh3hRUb643jmkZrn7zhIW', access_token_key='3173034274-Lj194JwBIoHIAMQDrUXvZw2YlFNsgNMYd4xVz1R',access_token_secret='pLQbEnbbr8YJRFmMvfoWdVwWwBfYN0UjSrBOHA01yC39M')

In [339]:
print api.VerifyCredentials()

{"created_at": "Thu Apr 16 14:50:48 +0000 2015", "default_profile": true, "friends_count": 1, "id": 3173034274, "lang": "en", "name": "Josh Malina", "profile_background_color": "C0DEED", "profile_background_image_url": "http://abs.twimg.com/images/themes/theme1/bg.png", "profile_background_tile": false, "profile_image_url": "https://pbs.twimg.com/profile_images/588716795504173056/NtsGGIaI_normal.jpg", "profile_link_color": "0084B4", "profile_sidebar_fill_color": "DDEEF6", "profile_text_color": "333333", "protected": false, "screen_name": "Joshua_R_Malina", "status": {"created_at": "Sat Sep 05 01:06:27 +0000 2015", "favorited": false, "id": 639967797724889088, "lang": "en", "retweeted": false, "source": "<a href=\"http://twitter.com\" rel=\"nofollow\">Twitter Web Client</a>", "text": "If anyone wanted 57,000 lines of weather + pollution data for Beijng (2008 - 2015): https://t.co/AJH3PfJ7WD", "truncated": false, "urls": {"https://t.co/AJH3PfJ7WD": "https://github.com/joshmalina/pollutio

In [346]:
tweets = [s.text for s in api.GetUserTimeline(screen_name='@BeijingAir')]

In [583]:
prev_vals = [int(x.split(';')[-2]) for x in tweets][0:5]
prev_vals

[80, 82, 80, 76, 72]

In [584]:
prev_vals.pop()

72

In [585]:
prev_vals

[80, 82, 80, 76]

In [586]:
prev_vals.insert(0, 4)

In [587]:
prev_vals

[4, 80, 82, 80, 76]

In [588]:
prev_vals.insert(0, 8888)

In [589]:
prev_vals

[8888, 4, 80, 82, 80, 76]

In [590]:
prev_vals.pop()

76

In [591]:
prev_vals

[8888, 4, 80, 82, 80]